In [138]:
### File Data
#sorted but not anlayzed files: /media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/Mishas_Demultiplexed/
#Take the first 500 files (2000 lines)

#### of all 4 file types - If you do it multiple times it just appends text lines to bottom of text editor
#(terminal)
#$head -n2000 N706_N504_R1.fastq >> ./10000Reads/N706_N504_R1.fastq
#$head -n2000 N706_N504_R2.fastq >> ./10000Reads/N706_N504_R2.fastq

#$head -n2000 N707_N505_R1.fastq >> ./10000Reads/N707_N505_R1.fastq
#$head -n2000 N707_N505_R2.fastq >> ./10000Reads/N707_N505_R2.fastq

#### check how many lines: (terminal)
#$wc -l N706_N504_R1.fastq

### can compress with gZip
#$gzip N706_N505_R1.fastq
#$gzip N706_N505_R2.fastq

In [139]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import numpy as np
import os
import sys
import gzip
import itertools
import operator
import subprocess
import twobitreader
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pysam

from uditas.uditas_replace_helpers import *



#  Pipeline1
# This, for the most part, is the UDITAS pipeline modified to allow for REPLACE targeting. 

## It is not possible to have UMI counts through LAM. So that info is not real. Ignore it.
## There is a second pipeline below to do global alignments to quantify large deletions, translocatins, concatomers, etc by aligning the sequence past the break site to the entire genome+targeting sequence.

### 1


Should also note that this is slightly different than the Tn5. It needs clipping of more sequence. We create false UMIS to satisfy the UDITAS pipeline by taking the first 5nt of the sequence. Also the adapter needs to be clipped. So a new function was made. Also the direction of read1:read2 is switched in order because of our choice of p5 and p7 attached to the gene specific sequence.


## Inputs


In [140]:
#things taken from original uditas args

check_plasmid_insertions = 1
ncpu = 4
window_size = 15
amplicon_window_around_cut = 1000
min_MAPQ = 5
min_AS = -180
process_AMP_seq_run = 0 #off

In [141]:
#Directory

#this is a minimal directory of only 300 files for quick debugging
#directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/Mishas_Demultiplexed_LAM'

directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/Mishas_Demultiplexed_LAM/10000Reads'

print(directory)

/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/Mishas_Demultiplexed_LAM/10000Reads


In [142]:
# number in list of which sheet to access in the sample_info.csv

#0 is LAM polb rev WT N706 N504 (row 0 in sheet)
#1 is LAM polb rev Targeting 50% N706 N505(row 1 in sheet)
#2 is LAM mCherryRev N707 N505, row2 in sheet'
#3 is LAM mCherryFWD   N708 N505 , row3 in sheet'


master_number = 2


#################################################################
#primer sequences for mispriming. Primer sequence + 12nt downstream 
#this will only be avialable for read1  (different than for Tn5 setup)

##### the sequences need to be capital for this to work########


primerlist = [['GGCCAAGCTGGAGCAGGAAATAGATGCACACG', 'this is for pipeline1 LAM- polb rev WT N706 N504 (row 0 in sheet)'],
              ['GGCCAAGCTGGAGCAGGAAATAGATGCACACG', 'this is for pipeline1 LAM polb rev Targeting 50% N706 N505(row 1 in sheet)'],
              ['TGCATCGCATTGTCTGAGTAGGTGTCATTCTATTC', 'this is for pipeline1 LAM mCherryfwd N707 N505, row2 in sheet'],
              ['CCTCCACGTCACCGCATGTTAGCAGACTTCCT', 'this is for mCherryRev LAM  N708 N505 , row3 in sheet']]

                

primer_seq_plus_downstream = primerlist[master_number][0]          
print(primerlist[master_number][1], '\n')    
print('primer_seq_plus_downstream', primer_seq_plus_downstream)


#check what is printed below!!!

########################################################################
# Function to make the 'amplicon_info' list. Taking the line of our experiment csv file

def get_csv_data(dir_sample, line_of_data_from_csv):
    sample_info_filename = os.path.join(dir_sample, 'sample_info.csv')
    experiments = pd.read_csv(sample_info_filename)
    return experiments.loc[line_of_data_from_csv]


#the amplicon info is related to the line on the csv file. It is indexed from 0.
#it is on my extera space so need to make sure that is mounted. (first thing to check if it throws and error)


amplicon_info = get_csv_data(directory, master_number)

###########check waht is printed below #######
amplicon_info

this is for pipeline1 LAM mCherryfwd N707 N505, row2 in sheet 

primer_seq_plus_downstream TGCATCGCATTGTCTGAGTAGGTGTCATTCTATTC


NGS_req-ID                                                                     A000H2GWGF
name                                                                        LAM_Pol50_bpA
Sample                                                                               Polb
description                                         MiniSeq_K562_Polb_mCherrFwd_50per_LAM
Control sample (Y/N)                                                                    N
Notes                                                                                 NaN
Dilution                                                                              NaN
Cell name_type                                                                       K562
UMI_Len                                                                        NNNNNNNNNN
IndexI7Primer                                                                         NaN
I7_Index_ID                                                          P7_N707_SBS12nextera
index_I1  

In [143]:
# Assign the file_genome_2bit location. This is needed for pulling sequence from the referene genome by location
assembly = amplicon_info['genome']
file_genome_2bit = os.path.join('/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes', assembly + '.2bit')
print(file_genome_2bit)

# BOWTIE2_INDEXES are needed for global alignments
#not sure if this will work
#normally in bash: export BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes
#check in bash: > ECHO $GENOMES_2BIT

%env BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes

/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes/hg38.2bit
env: BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes


### Pull out the UMI info and clip the adapter sequence


In [144]:
umi_joining(directory, amplicon_info)

number of reads: 500
number of unique umis 500


### Discard Mispriming Reads
When you put a universal primer on the ends of everything, every mispriming event will amplify. An effect we normally don't deal with. I did nested PCR to reduce this. However 85% of the alignments in the UDITAS data I looked at seemed to be mispriming. They did all their blasting and analysis before removing mispriming. But to save computational power and remove error early on I will discard mispriming events. 
They discard these only for plasmid alignments analyze_alignments_plasmid for some reason which comes from the bam file.

### read 1 and 2 are switched. By that I mean the gene specific primer is attached to the P7 sequence instead of P5

In [145]:
correct_priming_lam(directory, amplicon_info, primer_seq_plus_downstream)

total_read_number 500
reads_with_good_priming 470
reads_with_bad_priming 30
perfect matched 444
good but mismatched 26


In [146]:
#Running the trimming

trim_fastq(directory, amplicon_info, 0)

In [147]:
# Running the reference plasmid creation
create_plasmid_reference(directory, amplicon_info)

In [148]:
#Test the get_reaction_type is working
get_reaction_type(amplicon_info)

'replace'

In [149]:
#Make the amplicons by calling the create_amplicon function
create_amplicon(directory, amplicon_info, file_genome_2bit)

In [150]:
#try out the alignment to the plasmid
align_plasmid_local(directory, amplicon_info, ncpu=4)


In [151]:
#extract the unmapped reads
extract_unmapped_reads_plasmid(directory, amplicon_info)

In [152]:
#run the plasmid analysis to coudn plasmid integration events
result_plasmid_df = analyze_alignments_plasmid(directory, amplicon_info, min_MAPQ, file_genome_2bit, True)
result_plasmid_df

,target_plus_plasmid_total_reads,target_plus_plasmid_total_reads_collapsed,plasmid_only_total_reads,plasmid_only_total_reads_collapsed
0,0,0,2,2


In [153]:
#align against our suite of amplicons
align_amplicon(directory, amplicon_info, check_plasmid_insertions, ncpu)

In [154]:
#this will extract unmap reads new folder (files that did not align to the predicted structural variants)
extract_unmapped_reads_amplicons(directory, amplicon_info)

## This section is to do the analysis of the amplicon alignments

##### need to check my UMI stuff

In [155]:
result_reads_in_all_amplicons_df = analyze_alignments_all_amplicons(directory, amplicon_info, min_MAPQ, min_AS)
result_reads_in_all_amplicons_df

number of reads before mispriming cleanup 500
number of unique umis before mispriming cleanup 500


,all_amplicons_total_reads,all_amplicons_total_reads_collapsed
0,145,145


In [156]:
result_amplicon_df = analyze_alignments(directory, amplicon_info, window_size, amplicon_window_around_cut, min_MAPQ, min_AS)
result_amplicon_df

,wt_cut1_total_reads,wt_cut1_total_indels,wt_cut1_total_deletions,wt_cut1_total_insertions,wt_cut1_total_reads_collapsed,wt_cut1_total_indels_collapsed,wt_cut1_total_deletions_collapsed,wt_cut1_total_insertions_collapsed,wt_cut2_total_reads,wt_cut2_total_indels,...,1a_1a_cut1_total_insertions_collapsed,2b_2b_cut1_total_reads,2b_2b_cut1_total_indels,2b_2b_cut1_total_deletions,2b_2b_cut1_total_insertions,2b_2b_cut1_total_reads_collapsed,2b_2b_cut1_total_indels_collapsed,2b_2b_cut1_total_deletions_collapsed,2b_2b_cut1_total_insertions_collapsed,median_fragment_size
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,320.0


### This is the Global alignment of the remaining reads

#### What I need here is to modify it so that 

In [157]:
#Run the allignment
assembly = amplicon_info['genome']
align_genome_global(directory, amplicon_info, assembly)

In [158]:
#Do the global alignment analysis
results_genome_global_df = analyze_alignments_genome_global(directory, amplicon_info, min_MAPQ, min_AS,  file_genome_2bit)
results_genome_global_df



the number of alignments that were attempted to align 4


,genomewide_total_reads,genomewide_total_reads_collapsed,genomewide_target_only_reads,genomewide_target_only_reads_collapsed
0,2,2,0,0


### Summerize data Pipeline1


In [159]:
read_count = pd.DataFrame()
read_count.loc[0,'read_count'] = count_reads(directory, amplicon_info)
read_count.loc[0,'read_count']

470.0

In [160]:
#summerize the data

summary_all_alignments = get_summary_all_alignments(directory, amplicon_info, read_count.loc[[0]], result_plasmid_df, result_reads_in_all_amplicons_df, results_genome_global_df)
summary_all_alignments

,read_count,target_plus_plasmid_total_reads,target_plus_plasmid_total_reads_collapsed,plasmid_only_total_reads,plasmid_only_total_reads_collapsed,all_amplicons_total_reads,all_amplicons_total_reads_collapsed,genomewide_total_reads,genomewide_total_reads_collapsed,genomewide_target_only_reads,genomewide_target_only_reads_collapsed,total_aligned,total_aligned_collapsed,percent_aligned,percent_aligned_all_amplicons
0,470.0,0,0,2,2,145,145,2,2,0,0,149,149,31.702128,97.315436


In [169]:
results_alignments_junction = pd.concat([result_amplicon_df, result_plasmid_df], axis=1)
sample_info_filename = os.path.join(directory, 'sample_info.csv')

experiments = pd.read_csv(sample_info_filename)

N7 = amplicon_info['index_I1']
N5 = amplicon_info['index_I2']

results_summary = summarize_results(results_alignments_junction)
#this outputs it in a directory above the N501_701 folder as a summary file. Need to label it or it will be overwritten by next tone
results_summary_with_experiments = pd.concat([experiments, results_summary], axis=1)

results_summary_with_experiments.to_excel(os.path.join(directory, N7 + '_' + N5, N7 + '_' + N5+'_'+'results_summary.xlsx'))

results_pivot = melt_results_lam(results_summary_with_experiments)
results_pivot.to_excel(os.path.join(directory, N7 + '_' + N5, N7 + '_' + N5+'_'+'results_summary_pivot_no_UMI_collapse.xlsx'))
results_pivot

,NGS_req-ID,name,Sample,description,Control sample (Y/N),Notes,Dilution,Cell name_type,UMI_Len,IndexI7Primer,...,replace_rev_cut2_total_reads_collapsed_percent,doner_tail_tail_cut1_total_reads_collapsed_percent,doner_head_tail_cut1_total_reads_collapsed_percent,doner_head_head_cut1_total_reads_collapsed_percent,1a_1a_cut1_total_reads_collapsed_percent,2b_2b_cut1_total_reads_collapsed_percent,target_plus_plasmid_total_reads_collapsed_percent,plasmid_only_total_reads_collapsed_percent,Type,Percent Editing
0,A000H2GWGF,LAM_WT_pbR,WT,MiniSeq_K562_Polb_Rev_WT_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,0.0,0.0,66.666667,0.0,0.0,0.0,0.0,3.508772,wt_cut1_total_reads_percent,0.000000
1,A000H2GWGF,LAM_Pol50_pbR,Polb,MiniSeq_K562_Polb_Rev_50per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
2,A000H2GWGF,LAM_Pol50_bpA,Polb,MiniSeq_K562_Polb_mCherrFwd_50per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
3,A000H2GWGF,LAM_Pol50_mCherRev,Polb,MiniSeq_K562_Polb_mCherrRev_50per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
4,A000H2GWGF,LAM_Pol80_pbR,Polb,MiniSeq_K562_Polb_Rev_80per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
5,A000H2GWGF,LAM_Pol80_bpA,Polb,MiniSeq_K562_Polb_mCherrFwd_80per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
6,A000H2GWGF,LAM_Pol80_mCherRev,Polb,MiniSeq_K562_Polb_mCherrRev_80per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut1_total_reads_percent,NaN
7,A000H2GWGF,LAM_WT_pbR,WT,MiniSeq_K562_Polb_Rev_WT_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,0.0,0.0,66.666667,0.0,0.0,0.0,0.0,3.508772,wt_cut2_total_reads_percent,0.000000
8,A000H2GWGF,LAM_Pol50_pbR,Polb,MiniSeq_K562_Polb_Rev_50per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut2_total_reads_percent,NaN
9,A000H2GWGF,LAM_Pol50_bpA,Polb,MiniSeq_K562_Polb_mCherrFwd_50per_LAM,N,NaN,NaN,K562,NNNNNNNNNN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wt_cut2_total_reads_percent,NaN


# Pipeline2
#  This is a second pipeline. Different than Uditas but built in a simliar form. 
## It uses the read distal to the gene specic primer. This read covers the region downstream of the break. After clipping off anything remaing upstream of the break we make a global alignment to look at genomic integrations and translocations. This also quantifies the alignments.


### 0. Use correct_prime function in pipeline1 to arrive at reads that were correctly primed.
### 1. Trim correctly primed reads to the break site  
### 2. Only use a single end of the pair end reads (distal gene specific primer)
### 3. Use bowtie2 genome that has the targeting seq in it added
### 4. Do global end-to-end alignment
### 5. Do a local alignment
### 6. process bam file and pull out the high mapQ reads

The idea here is to trim the sequence up to the cut on both reads and then do a global alignment end-to-end agasint the human genome.

The idea here is to take the files that are 'trimmed' and 'correct priming' checking. Then to do another round of trimming to get down to the cut site. I need to make another folder for 'trimmed_to_cut'

For Tn5 setup the 'Read 1' contains the sequence away from the gene specific primer. So it is the read we should concentration on if we don't do paired end.

For LAM the 'Read 2' is the the sequence distal to the gene specific primer

In [162]:
#This is for the sequence to the breaksite for the pipeline of genomic alignment 
# make sure the sequences are capital

print()
seq_to_breaksite = [['GGCCAAGCTGGAGCAGGAAATAGATGCACACGGAGGAAATGGTAGTGGAGTTCAGAGGAGGGAGAGGTTCTTTCTGCCTGGG', 'this is for pipeline1 LAM- polb rev WT N706 N504 (row 0 in sheet)'],
                    ['GGCCAAGCTGGAGCAGGAAATAGATGCACACGGAGGAAATGGTAGTGGAGTTCAGAGGAGGGAGAGGTTCTTTCTGCCTGGG','this is for pipeline1 LAM polb rev Targeting 50% N706 N505(row 1 in sheet)'],
                    ['TGCATCGCATTGTCTGAGTAGGTGTCATTCTATTCTGGGGGGTGGGGTGGGGCAGGACAGCAAGGGGGAGGATTGGGAAGAGAATAGCAGGCATGCTGGGGGGATCCCTGCAGGAGCTCGTCGACCCATGGGGGCCCGCCCCAACTGGGGTAACCTTTGGGCTCCCCGGGCGCGACTAGTGAATTCAGATCTGATATCTCTAGAAGTCCTGGG','this is for pipeline1 LAM mCherryFwd N707 N505, row2 in sheet'],
                    ['CCTCCACGTCACCGCATGTTAGCAGACTTCCTCTGCCCTCAATCTTTAAGAAAAAAAAAAGTCTAACAATGATTTAGGAATGCTTTGAGGACTTAAATGATCTTATTGGAAACATACCAGTCTGCTAAAAGACTAATTTTGTGTGGGTCA', 'this is for mCherryrev LAM  N708 N505 , row3 in sheet']]


seq_primer_to_breaksite = seq_to_breaksite[master_number][0]
print(seq_to_breaksite[master_number][1],"\n")    
print('seq_to_breaksite', seq_primer_to_breaksite, '\n')


#check amplicon info and directory
print(directory)
amplicon_info







this is for pipeline1 LAM mCherryFwd N707 N505, row2 in sheet 

seq_to_breaksite TGCATCGCATTGTCTGAGTAGGTGTCATTCTATTCTGGGGGGTGGGGTGGGGCAGGACAGCAAGGGGGAGGATTGGGAAGAGAATAGCAGGCATGCTGGGGGGATCCCTGCAGGAGCTCGTCGACCCATGGGGGCCCGCCCCAACTGGGGTAACCTTTGGGCTCCCCGGGCGCGACTAGTGAATTCAGATCTGATATCTCTAGAAGTCCTGGG 

/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/Mishas_Demultiplexed_LAM/10000Reads


NGS_req-ID                                                                     A000H2GWGF
name                                                                        LAM_Pol50_bpA
Sample                                                                               Polb
description                                         MiniSeq_K562_Polb_mCherrFwd_50per_LAM
Control sample (Y/N)                                                                    N
Notes                                                                                 NaN
Dilution                                                                              NaN
Cell name_type                                                                       K562
UMI_Len                                                                        NNNNNNNNNN
IndexI7Primer                                                                         NaN
I7_Index_ID                                                          P7_N707_SBS12nextera
index_I1  


## Need to make an new bowtie2 index file that includes targeting for alignment agains the whole genome so it is all in one sheet together. 

### Other option would be to align it to the amplicon, extract unaligned files and then align to the genome but seems cleaner this way. 
#### Ideally every sequence is unique between the targeting vector and genome.


1. Build your fastas of interest and label .fa files.
    1. You need fasta of hg38 or reference genome. You can pull this from downloaded bowtie indexed sampels and then use the following command to turn the index into a fasta file: bowtie2-inspect hg38 > hg38.fa   
    2. Put all the fasta files in the same folder
2. index the files with bowtie
    1. use the command bowtie2-build -f pe038_mc.fa,hg38.fa -p hg38_plus_targetvector
    2. this has the -p to make it take less ram in my case.
    3. In this case it adds the hg38 and the minicircle targeting file together
    4. I have a Intel® Core™ i7-5500U CPU @ 2.40GHz × 4 with 15.1 GiB ram and it required about 13.8 gigs of ram and 2 hours to do a hg38+small fasta index
    5. be sure to pay attention to the name of the new indexed file. "hg38_plus_targetvector" in example above. Add it to the sample_info.csv sheet. Under the tab 'genome_plus_targeting'.
    6. you can check it indexed correcctly: bowtie2-inspect -s hg38_plus_targetvector
    

    



In [163]:
# Assign the file_genome_2bit location. This is needed for pulling sequence from the referene genome by location
#need a new build
assembly_plus_targetvector = amplicon_info['genome_plus_targeting']
file_genome_2bit_plus_target = os.path.join('/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes', assembly_plus_targetvector + '.2bit')
print(file_genome_2bit_plus_target)

#Needs to have a bowtie build in which the targeting vector is included!!

%env BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes

#check the env variables
#%env

/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes/hg38_plus_targetvector.2bit
env: BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes


In [164]:
#Trim the break up to the cut site
trim_fastq_to_break(directory, amplicon_info, seq_primer_to_breaksite, lam_or_tn5='lam')

lam analysis


In [165]:
#this aligns everything after the break to end-to-end 
align_afterbreak_end_to_end_genome_global(directory, amplicon_info, assembly_plus_targetvector, lam_or_tn5='lam', keep_sam=0)

lam analysis
sam file deleted


In [166]:
#this aligns everything in the local format wehre it can soft clip the ends
align_afterbreaks_genome_local(directory, 1, amplicon_info, assembly_plus_targetvector, lam_or_tn5 ='lam', keep_sam=0)

lam analysis
sam file deleted
